In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import NBEATSModel
from fancyimpute import KNN
import numpy as np
import pandas as pd

/efs/litmus-server/users/yguptgau/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-14 07:03:48 prophet.plot ERROR: Importing plotly failed. Interactive plots will not work.
2022-06-14 07:03:52.061978: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
df = pd.read_csv('../Data/Dummy_data_1week_7Dec.csv')
df = df.set_index(['lat_grid','long_grid'])

In [4]:
data = np.array(df)
test_portion = 6
data[:,:-test_portion] = np.where(data[:,:-test_portion] == 0, np.nan, data[:,:-test_portion])
knn_imputer = KNN()
data[:,:-test_portion] = knn_imputer.fit_transform(pd.DataFrame(data[:,:-test_portion]))

Imputing row 1/185 with 0 missing, elapsed time: 0.013
Imputing row 101/185 with 2 missing, elapsed time: 0.033
[KNN] Warning: 72/6660 still missing after imputation, replacing with 0


In [7]:
model = NBEATSModel(input_chunk_length=6, output_chunk_length=6, random_state=42, pl_trainer_kwargs={"accelerator": "gpu",
                                                                                                       "gpus": [1]})


In [8]:
model.fit([TimeSeries.from_values(train_series[:-test_portion]) for train_series in data], epochs=2, verbose=True)
pred_air = model.predict(series=[TimeSeries.from_values(time_series[:-test_portion]) for time_series in data], n=test_portion)


[2022-06-14 09:13:33,018] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 4625 samples.
[2022-06-14 09:13:33,018] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 4625 samples.
2022-06-14 09:13:33 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 4625 samples.
[2022-06-14 09:13:33,076] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-06-14 09:13:33,076] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-06-14 09:13:33 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
[2022-06-14 09:13:33,077] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Light

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████| 145/145 [00:09<00:00, 14.67it/s, loss=2.56e+03, train_loss=3.38e+3]


2022-06-14 09:13:55 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, -780.39it/s]


In [10]:
predicted_series = model.predict(series=[TimeSeries.from_values(time_series[:-test_portion]) for time_series in data], n=test_portion)

2022-06-14 09:13:59 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, -784.92it/s]


In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
actual_test = data[:,-test_portion:]
predicted_test = np.array([series.values().reshape(-1,) for series in predicted_series])         
rmse = sqrt(mean_squared_error(actual_test[actual_test>0], predicted_test[actual_test>0]))
rmse

44.51028238167593

# Results of baseline -NBeats

In [13]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
days = ['7Jan', '10Jan', '15Dec', '7Dec', '24Jan', '15Nov', '28Nov', '20Nov']
filename = ['../Data/Dummy_data_1week_'+ day +'.csv' for day in days]
test_rmse = []
for file in filename:
    df = pd.read_csv(file)
    df = df.set_index(['lat_grid','long_grid'])
    train_size = 36
    test_portion = 6
    data = np.array(df)
    data[:,:-test_portion] = np.where(data[:,:-test_portion] == 0, np.nan, data[:,:-test_portion])
    knn_imputer = KNN()
    data[:,:-test_portion] = knn_imputer.fit_transform(pd.DataFrame(data[:,:-test_portion]))
    model = NBEATSModel(input_chunk_length=6, output_chunk_length=6, random_state=42, pl_trainer_kwargs={"accelerator": "gpu",
                                                                                                       "gpus": [1]})
    model.fit([TimeSeries.from_values(train_series[:-test_portion]) for train_series in data], epochs=2, verbose=True)
    pred_air = model.predict(series=[TimeSeries.from_values(time_series[:-test_portion]) for time_series in data], n=test_portion)
    predicted_series = model.predict(series=[TimeSeries.from_values(time_series[:-test_portion]) for time_series in data], n=test_portion)
    actual_test = data[:,-test_portion:]
    predicted_test = np.array([series.values().reshape(-1,) for series in predicted_series])         
    rmse = sqrt(mean_squared_error(actual_test[actual_test>0], predicted_test[actual_test>0]))
    print(rmse)
    test_rmse.append(rmse)

[2022-06-14 09:16:38,173] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 4025 samples.
[2022-06-14 09:16:38,173] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 4025 samples.
2022-06-14 09:16:38 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 4025 samples.
[2022-06-14 09:16:38,232] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-06-14 09:16:38,232] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-06-14 09:16:38 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
[2022-06-14 09:16:38,234] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Light

Imputing row 1/161 with 31 missing, elapsed time: 0.011
Imputing row 101/161 with 5 missing, elapsed time: 0.025
[KNN] Warning: 108/5796 still missing after imputation, replacing with 0


2022-06-14 09:16:38 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
2022-06-14 09:16:38 pytorch_lightning.callbacks.model_summary INFO: 
  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | stacks    | ModuleList | 6.1 M 
-----------------------------------------
6.1 M     Trainable params
1.3 K     Non-trainable params
6.1 M     Total params
48.434    Total estimated model params size (MB)


Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████| 126/126 [00:09<00:00, 13.24it/s, loss=2.35e+03, train_loss=2.14e+3]


2022-06-14 09:16:57 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


...

Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████| 120/120 [00:09<00:00, 13.11it/s, loss=1.64e+03, train_loss=1.2e+3]


2022-06-14 09:19:24 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, -768.78it/s]


2022-06-14 09:19:24 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, -776.41it/s]
62.11738106885674


In [15]:
out = pd.DataFrame()
out['rmse'] = test_rmse
out['day'] = days
out.to_csv('NBeats_results.csv')

In [16]:
out

,rmse,day
0,34.562827,7Jan
1,51.002434,10Jan
2,59.244561,15Dec
3,44.510282,7Dec
4,70.699566,24Jan
5,157.184155,15Nov
6,64.645340,28Nov
7,62.117381,20Nov


In [19]:
out['rmse'].mean()

67.99581808812304